In [3]:
import os
import pandas as pd

1. need to parse useful columns
2. hospitilizations/deaths targets are missing
3. compare delta vs omicron waves
    * Delta and Omicron not correlated the same in hosp, split dataset? 
    * numbers for hosp include those there for flagged for COVID, not recessarily there because of COVID
4. how to handle the metrics and normalization. 
    * metric for hosp will be change in hospitlizations? Like 10 increase? % increase
    
**NEED DATA ON:**
1. hospitilizations, vaccine rates, Omicron/Delta/Alpha proportions

## Google Data
* missing data in most columns. train quick RNN on seasonality data

In [87]:
# can also download directly from source: https://storage.googleapis.com/covid19-open-data/v3/ - needs to download though
file = 'mobility.csv'  #epidemiology.csv, mobility.csv
df = pd.read_csv(os.path.join('data', file))

In [92]:
# extract from location_key all Canadian entries
is_CA = df['location_key'].apply(lambda x: x.startswith('CA_MB') if isinstance(x, str) else False)
df_CA = df[is_CA]

In [93]:
df_CA

,date,location_key,mobility_retail_and_recreation,mobility_grocery_and_pharmacy,mobility_parks,mobility_transit_stations,mobility_workplaces,mobility_residential
1362007,2020-02-15,CA_MB,5.0,2.0,20.0,1.0,1.0,0.0
1362008,2020-02-16,CA_MB,10.0,0.0,26.0,-4.0,-4.0,-2.0
1362009,2020-02-17,CA_MB,-19.0,-24.0,60.0,-37.0,-69.0,17.0
1362010,2020-02-18,CA_MB,3.0,6.0,4.0,-5.0,-1.0,1.0
1362011,2020-02-19,CA_MB,0.0,-1.0,0.0,-9.0,0.0,1.0
...,...,...,...,...,...,...,...,...
1362719,2022-01-27,CA_MB,-13.0,2.0,-12.0,-54.0,-20.0,10.0
1362720,2022-01-28,CA_MB,-15.0,-4.0,-22.0,-55.0,-20.0,10.0
1362721,2022-01-29,CA_MB,-13.0,-3.0,-9.0,-47.0,-4.0,6.0
1362722,2022-01-30,CA_MB,-16.0,-2.0,-12.0,-48.0,-12.0,5.0


In [64]:
# use simple RNN to predict the missing data
df_CA[df_CA.columns[2]]

1359856     4.0
1359857    13.0
1359858   -12.0
1359859    -1.0
1359860     1.0
           ... 
1369662   -21.0
1369663   -28.0
1369664   -30.0
1369665   -35.0
1369666   -29.0
Name: mobility_retail_and_recreation, Length: 9811, dtype: float64

## Government of Canada Data
* numconf - number of confirmed cases - cumulative total, not helpful
* numdeaths - number of deaths - bad lag time
* numtoday - number of total counts from last update subtracted from total counts of current update
* numactive - number of active cases - useful!
* rateactive - number of active cases per 100k - usefull!

In [58]:
file = 'covid19-download.csv'  #epidemiology.csv, mobility.csv
df = pd.read_csv(os.path.join('data', file))

In [59]:
df[df['prname'] == 'Manitoba'].sort_values(by='date')

,pruid,prname,prnameFR,date,update,numconf,numprob,numdeaths,numtotal,numtested,...,ratedeaths_last14,numtotal_last7,ratetotal_last7,numdeaths_last7,ratedeaths_last7,avgtotal_last7,avgincidence_last7,avgdeaths_last7,avgratedeaths_last7,raterecovered
60,46,Manitoba,Manitoba,2020-03-11,NaN,0,0.0,0.0,0,352.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00
75,46,Manitoba,Manitoba,2020-03-12,NaN,0,1.0,0.0,1,352.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00
90,46,Manitoba,Manitoba,2020-03-13,NaN,1,2.0,0.0,3,352.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00
105,46,Manitoba,Manitoba,2020-03-14,NaN,1,3.0,0.0,4,837.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00
120,46,Manitoba,Manitoba,2020-03-15,NaN,4,0.0,0.0,4,837.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10410,46,Manitoba,Manitoba,2022-01-30,0.0,120139,NaN,1543.0,120139,NaN,...,6.21,4542.0,328.23,51.0,3.69,649.0,46.89,7.0,0.53,0.00
10425,46,Manitoba,Manitoba,2022-01-31,1.0,120644,NaN,1562.0,120644,NaN,...,7.15,4511.0,325.99,47.0,3.40,644.0,46.57,7.0,0.49,0.00
10440,46,Manitoba,Manitoba,2022-02-01,1.0,121131,NaN,1569.0,121131,NaN,...,7.44,4361.0,315.15,48.0,3.47,623.0,45.02,7.0,0.50,0.00
10455,46,Manitoba,Manitoba,2022-02-02,1.0,121655,NaN,1576.0,121655,NaN,...,7.08,4260.0,307.86,52.0,3.76,609.0,43.98,7.0,0.54,3.61


In [60]:
df.tail()[df.columns[:10]]

,pruid,prname,prnameFR,date,update,numconf,numprob,numdeaths,numtotal,numtested
10477,60,Yukon,Yukon,2022-02-03,1.0,3168,NaN,18.0,3168,NaN
10478,61,Northwest Territories,Territoires du Nord-Ouest,2022-02-03,1.0,6363,NaN,17.0,6363,NaN
10479,62,Nunavut,Nunavut,2022-02-03,1.0,1891,NaN,5.0,1891,NaN
10480,99,Repatriated travellers,Voyageurs rapatriés,2022-02-03,NaN,13,NaN,0.0,13,NaN
10481,1,Canada,Canada,2022-02-03,NaN,3096217,NaN,34381.0,3096217,NaN
